In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from facerec_utils import *
from inception_blocks_v2 import *
import pyttsx3
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.inf)

Using TensorFlow backend.


In [2]:
face_recog_model = faceRecoModel(input_shape=(3, 96, 96))

In [3]:
print("Total Params:", face_recog_model.count_params())

Total Params: 3743280


In [4]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
   
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = pos_dist - neg_dist + alpha
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [5]:
def onStart(): 
    print('starting') 

def onWord(name, location, length): 
    print('word', name, location, length) 

def onEnd(name, completed): 
    print('finishing', name, completed)
engine = pyttsx3.init()

In [6]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 528.14307


##  Loading the pre-trained model


In [7]:
face_recog_model.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(face_recog_model)

## Applying the model

In [26]:
database = {}
database["sanjeev"] = img_to_encoding("images/sanjeev.jpg", face_recog_model)
database["aishwarya rai"] = img_to_encoding("images/aishwarya rai.jpg", face_recog_model)
database["a r rahman"] = img_to_encoding("images/a r rahman.jpg", face_recog_model)

In [27]:
def verify(image_path, identity, database, model):
   
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding - database[identity])
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.95:
        print("It's " + str(identity) + ", welcome home!")
        engine.connect('started-utterance', onStart) 
        engine.connect('started-word', onWord) 
        engine.connect('finished-utterance', onEnd) 
        sen = "Welcome back home"+ str(identity);
        engine.say(sen)
        engine.runAndWait() 
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        engine.connect('started-utterance', onStart) 
        engine.connect('started-word', onWord) 
        engine.connect('finished-utterance', onEnd) 
        sen = "It's not"+ str(identity +", please go away");
        engine.say(sen)
        engine.runAndWait()
        door_open = False
        
    return dist, door_open

In [28]:
verify("images/ex1.jpg", "aishwarya rai", database, face_recog_model)

It's aishwarya rai, welcome home!
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True

(0.41985664, True)

In [29]:
verify("images/ex2.jpg", "a r rahman", database, face_recog_model)

It's a r rahman, welcome home!
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 13 5
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 19 1
word None 21 6
word None 21 6
word None 21 6
word None 21 6
word None 21 6
word None 21 6
word None 21 6
word None 21 6
word None 21 6
word None 21 6
word No

(0.68740714, True)

In [30]:
verify("images/ex3.jpg", "sanjeev", database, face_recog_model)

It's sanjeev, welcome home!
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True


(0.4599099, True)

In [31]:
def recog(image_path, database, model):
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding - db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name


    
    if min_dist > 0.95:
        print("Not in the database.")
        engine.connect('started-utterance', onStart) 
        engine.connect('started-word', onWord) 
        engine.connect('finished-utterance', onEnd) 
        sen = "Not in the database.";
        engine.say(sen)
        engine.runAndWait()
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        engine.connect('started-utterance', onStart) 
        engine.connect('started-word', onWord) 
        engine.connect('finished-utterance', onEnd) 
        sen = "Welcome back home"+ str(identity);
        engine.say(sen)
        engine.runAndWait()
        
    return min_dist, identity

In [32]:
recog("images/ex1.jpg", database, face_recog_model)

it's aishwarya rai, the distance is 0.41985664
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
finishing

(0.41985664, 'aishwarya rai')

In [33]:
recog("images/ex2.jpg", database, face_recog_model)

it's aishwarya rai, the distance is 0.6103996
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 13 13
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 27 3
word None 2

(0.6103996, 'aishwarya rai')

In [34]:
recog("images/ex3.jpg", database, face_recog_model)

it's sanjeev, the distance is 0.4599099
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 0 7
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 8 4
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
word None 13 11
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishing None True
finishin

(0.4599099, 'sanjeev')